# BitChute Scraper Examples

In [1]:
import bitchute as bc

In [2]:
# Initialize the BitChute API client
api = bc.BitChuteAPI(verbose=False)

## Get Platform Recommendations

In [3]:
# Retrieve videos listed under "Trending"
trending = api.get_trending_videos('day', include_details=False, limit=2)
print(f"Success! Got {len(trending)} videos")
print()
trending.head(2)

Success! Got 2 videos



,id,title,description,view_count,duration,thumbnail_url,video_url,channel_id,channel_name,category,upload_date,hashtags,is_short,like_count,dislike_count,media_url,media_type,local_thumbnail_path,local_video_path
0,hgAnl3vW64W9,"Ep 3688b - [DS] Narrative Failed, Trump Beat T...",🍃 Help take years off the clock with Collagen...,8482,59:38,https://static-3.bitchute.com/live/cover_image...,https://www.bitchute.com/video/hgAnl3vW64W9/,n78PbEkvWx2g,X22report,,2025-07-18T20:48:21.230415Z,[],False,0,0,,,,
1,p4Uq2Yq8A7GP,"WTC-7, A FIFTH PLANE & THE WHOLE TRUTH -- Rich...",Help take years off the clock with Collagen🍃\n...,7286,1:17:51,https://static-3.bitchute.com/live/cover_image...,https://www.bitchute.com/video/p4Uq2Yq8A7GP/,nLPcg68RnP97,SGT Report,,2025-07-19T02:11:53.688317Z,[],False,0,0,,,,


In [4]:
# Retrieve videos listed under "Fresh"
popular = api.get_popular_videos(include_details=False)
print(f"Success! Got {len(popular)} videos")
print()
popular.head(2)

Success! Got 197 videos



,id,title,description,view_count,duration,thumbnail_url,video_url,channel_id,channel_name,category,upload_date,hashtags,is_short,like_count,dislike_count,media_url,media_type,local_thumbnail_path,local_video_path
0,9NxPS1bAK5km,"Andkon's Reich - Debunking The Holocaust, Jul ...",,8,5:31:22,https://static-3.bitchute.com/live/cover_image...,https://www.bitchute.com/video/9NxPS1bAK5km/,kDHBE5vylTdI,Jim Rizoli,,2025-07-19T16:22:02.945637Z,[],False,0,0,,,,
1,xjtBoqn48Jhp,3 men try to set a house on fire but light up ...,,41,0:27,https://static-3.bitchute.com/live/cover_image...,https://www.bitchute.com/video/xjtBoqn48Jhp/,qkoZx4ASWYHq,UNDERSTANDING THE LIES,,2025-07-19T16:19:53.435044Z,[],False,0,0,,,,


In [5]:
# Retrieve videos listed under "Shorts"
short = api.get_short_videos()
print(f"Success! Got {len(short)} videos")
print()
short.head(2)

Success! Got 182 videos



,id,title,description,view_count,duration,thumbnail_url,video_url,channel_id,channel_name,category,upload_date,hashtags,is_short,like_count,dislike_count,media_url,media_type,local_thumbnail_path,local_video_path
0,XLtHfS4yuymJ,Le plus grand laboratoire français n'évite pas...,,4,0:47,https://static-3.bitchute.com/live/cover_image...,https://www.bitchute.com/video/XLtHfS4yuymJ/,I0yA5pMoOF7W,lababouche,,2025-07-19T16:21:37.271969Z,[],False,0,0,,,,
1,ZaDD-ifvApw,Maddox Voice Line With Chemicalized Salamander...,,3,0:27,https://static-3.bitchute.com/live/cover_image...,https://www.bitchute.com/video/ZaDD-ifvApw/,tD4JB6wIvKEK,TeamCortex,,2025-07-19T16:21:09.966995Z,[],False,0,0,,,,


In [6]:
# Retrieve videos listed under "Member Picked"
# CURRENTLY A POTENTIALLY INTERESTNIG DATAPOINT IS NOT INCLUDED IN THE RESULTS: date_liked. NEEDS ENHANCEMENT

member_picked = api.get_member_picked_videos(include_details=False)
print(f"Success! Got {len(member_picked)} videos")
print()
member_picked.head(2)

Success! Got 100 videos



,id,title,description,view_count,duration,thumbnail_url,video_url,channel_id,channel_name,category,upload_date,hashtags,is_short,like_count,dislike_count,media_url,media_type,local_thumbnail_path,local_video_path
0,bvorNsx5NF1K,Gaza Trump Genocide,Christian Zionism for Dummies\r\nhttps://gibra...,35,22:26,https://static-3.bitchute.com/live/cover_image...,https://www.bitchute.com/video/bvorNsx5NF1K/,OBgbkWHeNMQP,GodTaughtMeHow,,2025-07-19T11:01:19.078928Z,[],False,0,0,,,,
1,rF2O7LP2Tsb9,"Jim Rizoli - Auschwitz Fakery, Part 2, Jul 18,...",Jim exposes more fakery of the Auschwitz building,185,24:18,https://static-3.bitchute.com/live/cover_image...,https://www.bitchute.com/video/rF2O7LP2Tsb9/,kDHBE5vylTdI,Jim Rizoli,,2025-07-18T14:45:52.667714Z,[],False,0,0,,,,


In [ ]:
# Retrieve videos listed under "all"
all_videos = api.get_all_videos(include_details=False)
print(f"Success! Got {len(all_videos)} videos")
print()
all_videos.head(2)

In [ ]:
# Retrieve trending hashtags
hashtags = api.get_trending_hashtags()
print(f"Success! Got {len(hashtags)} hashtags")
print()
hashtags.head(2)

## Search the BitChute Platform

In [ ]:
# Search Videos by Query
#
# Options for `sensitivity` and `sort`:
# sensitivity: 'nsfw', 'nsfl' defaults to 'normal'
# sort: 'views', 'old' defaults to 'new'

search = api.search_videos('bitcoin', limit=10, sensitivity='normal', sort='new', include_details=True) 
print(f"Success! Got {len(search)} videos")
print()
search.head(2)

In [ ]:
# Retrieve Videos by Hashtag
videos = api.get_videos_by_hashtag('truth', limit=10, include_details=True)
videos

In [ ]:
# Search Channels by Query
#
# Options for `sensitivity`:
# sensitivity: 'nsfw', 'nsfl' defaults to 'normal'

channels = api.search_channels('truth', limit=10, sensitivity='normal', include_details=True)
print(f"Success! Got {len(channels)} channels")
print()
channels.head(2)

## Retrieve data for individual videos and channels
### Video

In [ ]:
video = api.get_video_info('JO_qKQUkTrg')
type(video)

### Channel

In [ ]:
# Get channel information:
# Get channel details
channel = api.get_channel_info('UnweMhJQFhlW')
type(channel)

In [ ]:
# Get channel videos
videos = api.get_channel_videos('UnweMhJQFhlW', limit=50000, include_details=True)
print(f"Success! Got {len(videos)} videos")
print()
videos.head(2)

## Downloading Thumbnails and Videos Example

In [ ]:
api = bc.BitChuteAPI(
        enable_downloads=True,
        thumbnail_folder="thumbnails",
        video_folder="videos",
        force_redownload=False,
        max_concurrent_downloads=3,
        verbose=True
    )

# CAREFUL: This will download the latest videos from the channel, which can be a lot of data.
videos = api.get_recent_videos(limit=4, include_details=True, download_thumbnails=True, download_videos=True)